# Sentence Classification

- 감성 분석(Sentiment Classification), 문서 쌍 분류(NLI)를 하기 위한 Bert Classification Model을 만듬


- 해당 모델에서는 데이터을 넣으면 감성 분석을 해야하는 경우에는 감성 분석을 문서 쌍 분류를 해야하는 경우에는 문서 쌍 분류를 하는 모델임


- 해당 모델을 사용하기 위해 전처리 함수를 만들어 놓았으나 데이터에 어느 정도의 전처리가 필요
 1. 데이터의 컬럼들은 id(있던 없던 상관없음), document_1, document_2(문서 쌍 분류의 경우), label(문자열 형식의 label도 괜찮음) 이렇게 4개를 권장함
 2. 또한 데이터 셋은 train_data, test_data 형식으로 권장함 val_data가 있는 경우
 train_data와 합쳐주거나 test_data와 합쳐주어도 괜찮음
 3. 전처리 함수에 들어가면 중복 데이터, 결측치, id컬럼을 제거함
 4. 컬럼의 이름을 document_1, document_2(문서 쌍 분류의 경우), str_label로 변경(label data가 문자열인 경우 str_label로 변경, 숫자열인 경우 label로 변경함)
 5. label 컬럼의 이름이 str_label인 경우 sklearn의 LabelEncoder를 통해 새로운 label 컬럼을 만들어 줌
 6. 최종적으로 데이터 변수 명은 train_data, test_data로 권장함


- 위의 전처리 과정을 거쳤다면 모델 학습을 위한 전처리가 필요함
    - 전처리 과정은 train_data의 전처리 과정과 test_data의 전처리 과정을 따로 분리 해놓음
    - test_data 전처리의 겨우 label이 없다는 가정하에 진행함
    - validation_data를 생성하기 때문에 test_data는 label이 없다고 가정함 
 


In [1]:
!pip3 install transformers

     |████████████████████████████████| 3.8 MB 7.4 MB/s 
     |████████████████████████████████| 6.5 MB 55.7 MB/s 
     |████████████████████████████████| 895 kB 72.7 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [79]:
import os
import json
import urllib.request

import pandas as pd
import numpy as np
import random
import time
import datetime

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader,TensorDataset,RandomSampler
from transformers import BertTokenizer,BertTokenizerFast
from transformers import get_linear_schedule_with_warmup
from transformers import BertConfig, BertForSequenceClassification, AdamW

from transformers import TextClassificationPipeline

from tqdm import tqdm
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [82]:
%cd /content/gdrive/My Drive/NLP

/content/gdrive/My Drive/NLP


In [3]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#  1. 데이터 전처리


In [4]:
def drop_na_and_duplicates(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df = df.drop('id',axis=1) if 'id' in df.columns else df
    try:
        df.columns = ['document_1','document_2','str_label'] if df.iloc[:,-1].dtypes == 'O' else ['document_1','document_2','label']
    except Exception:
        df.columns = ['document_1','str_label'] if df.iloc[:,-1].dtypes == 'O' else ['document_1','label']

    df['label'] = LabelEncoder().fit_transform(df['str_label'].tolist()) if 'str_label' in df.columns else df['label']

    return df

## 모델 불러오기 및 데이터 전처리 함수를 통해 전처리

### DATA 1. 네이버 영화 리뷰

- **긍정, 부정에 대한 감정 분석**

In [5]:
!git clone https://github.com/e9t/nsmc.git

train_data = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
train_data = drop_na_and_duplicates(train_data)

map_dict = {'0' : '부정', '1' : '긍정'}

dev_data = pd.read_csv("nsmc/ratings_test.txt", sep='\t')
dev_data = drop_na_and_duplicates(dev_data)

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [6]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 149995 
dev data 개수 : 49997
{'0': '부정', '1': '긍정'}


,document_1,label
128072,원본이 최고,1
65515,스릴감과 훈훈함이 있는 영화.,1
85046,굉장히 저평가되는 영화중 하나라고 생각함,1
118189,정말영화같은이야기 영화여서 영화같은이야기가 좋다,1
92366,계기도없는데 이상하다,0


,document_1,label
27461,꼭 보세요. 많은 생각이 듭니다.,1
21688,새벽에 봤는데 잠 안올 정도로 재밌었네요,1
41303,옥택연이연희 오그라드는 연기하며.. (어느정도냐면 이희준이 봐줄만한 수준) 제작진은...,0
21363,히치콕은역시천재최근스릴러들보다수준이높다,1
3630,이런 싱거운 드라큐라를 보았는가...음향소리 때문에 귀가 찢어질 듯,0


### DATA 2. 챗봇 데이터 

- **공포, 놀람, 중립, 분노, 슬픔, 행복, 혐오 ,공포 총 7개의 감정에 대한 감정 분석**

- 한국어 감정 정보가 포함된 단발성 대화 데이터셋

In [ ]:
chatbot_data = pd.read_excel('datas/korean_chatbot.xlsx')
chatbot_data = drop_na_and_duplicates(chatbot_data)

map_dict = dict(zip(chatbot_data['str_label'].unique(),chatbot_data['label'].unique()))

train_data, dev_data, train_label, dev_label = train_test_split(chatbot_data[['document_1','str_label']],chatbot_data['label'], test_size=0.2, random_state=42, shuffle=True)

train_data['label'] = train_label
dev_data['label'] = dev_label

In [ ]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 30822 
dev data 개수 : 7706
{'공포': 0, '놀람': 1, '분노': 2, '슬픔': 3, '중립': 4, '행복': 5, '혐오': 6}


,document_1,str_label,label
32135,"애플, 구글 세계최고",행복,5
17563,니들떄문에...내가 오늘...,슬픔,3
35996,이 골똥 보수 새기이들 이넘들은 OOO 같은 습숑구리들,혐오,6
36650,날씨랑 정치랑 죽이 딱 맞네ㅡ구라가 90프로~,혐오,6
31475,당근 김유정.진영이 딱~~!,행복,5


,document_1,str_label,label
25808,뭐 슈퍼맨으로 대상 빼곤 상 받을짓 없었지...,중립,4
19345,대놓고 친구한테 티는안내지만부러운건 어쩔수없네요,슬픔,3
22033,안녕..나의 미련이여..,슬픔,3
13557,10억엔에 목숨걸고 진짜 비굴하게 외교하네..일본대사 필요없고 외교부 문 닫아라 그냥,분노,2
10356,한국도 그렇지만 일본은더한거도같다,놀람,1


### DATA 3 카카오 브레인 데이터

- **document_1과 document_2의 문장 관계를 파악**
 - 문서 쌍 분류로 NLI라고 불림

- 간혹 BertForNextSentence 사용한다는 글이 있으니 나중에 꼭 해보자!

In [ ]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
train_data = train_snli.append(train_xnli)

dev_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

train_data = drop_na_and_duplicates(train_data)
dev_data = drop_na_and_duplicates(dev_data.append(test_data))

map_dict = dict(zip(train_data['str_label'].unique(),train_data['label'].unique()))

In [ ]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 941814 
dev data 개수 : 7500
{'neutral': 2, 'contradiction': 0, 'entailment': 1}


,document_1,document_2,str_label,label
25528,미 해병대는 작은 아이를 플라크와 케블러로 분장시켜 조준하는 법을 보여주었다.,해병이 개에게 케블러 옷을 입히고 있다,contradiction,0
623092,그리고 나는 계속 거절한다,"나는 그들에게 단 한 번, 한 번만 말했다.",contradiction,0
702503,두 마그 모두 그의 폭력성을 과소평가한다.,그는 교양이 있다.,contradiction,0
643248,오늘날의 큰 기부자들은 그들의 전임자들보다 더 실제적이라고 한 기사는 말한다.,오늘날의 기부자들은 그 어느 때보다도 더 보호받고 멀리 떨어져 있다.,contradiction,0
58210,남자가 권총을 쏘고 있다.,한 남자가 총기를 발사한다.,entailment,1


,document_1,document_2,str_label,label
1639,빈 라덴이 공격을 지휘한 것에 대한 증거를 찾기가 어렵습니다.,빈 라덴이 책임이 있다는 것을 증명하기는 어려웠다.,entailment,1
2922,마치 그녀가 나에게 준 것은 모두 상세하고 복잡했고 두 번째 것은 모두 단순했던 것...,그녀는 내게 두 가지 다른 버전을 주었다.,entailment,1
2435,"이러한 맥락에서 볼 때, 최근 미국의 옴니 가제터의 출판은 1,500,000 건의 ...",미국의 옴니 가제터는 포괄적이다.,contradiction,0
3561,Quincaeeras에 대한 연구의 대부분은 가족이 문화적 역사적 전통을 유지하고 ...,딸의 열 번째 생일은 문화적인 유대를 지속하는 수단이다.,contradiction,0
2224,프랑스 연구 결과는 기본적으로 작년에 미국에서 펜펜에 대한 1억 8천만 개의 처방전...,수백만의 사람들이 펜펜을 가져갔다.,entailment,1


# 2. 모델에 사용하기 위한 전처리

train data 전처리와 test data 전처리를 따로 분리해놓음

## 2-1. Data Preprocessing 

In [7]:
def preprocess(train_data, batch_size=32, method = 'train'):
    if 'document_2' not in train_data.columns:
        sent_1 = [str(sentence) for sentence in  train_data['document_1'].tolist()]
        batch_input = tokenizer(sent_1, padding=True, truncation=True)
    
    # NLI의 경우
    elif 'document_2' in train_data.columns:
        sent_1 = [str(sentence) for sentence in  train_data['document_1'].tolist()]
        sent_2 = [str(sentence) for sentence in  train_data['document_2'].tolist()]
        batch_input = tokenizer(sent_1, sent_2, padding=True, truncation=True)


    batch_input = {key : torch.tensor(value) for key, value in batch_input.items()}

    batch_input['labels'] = torch.tensor(train_data['label'].values)
    
    
    if method == 'train' or method == 'valid': 
        batch_input['labels'] = torch.tensor(train_data['label'].values)
        dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'], batch_input['labels'])
        if method == 'train':
            sampler = RandomSampler(dataset)
            dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
        elif method == 'valid':
            dataloader = DataLoader(dataset, batch_size=batch_size)
            
    elif method == 'test':
        dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'])
        dataloader = DataLoader(dataset, batch_size=batch_size)
        
    return dataloader

# 3. model 

## 1. model train

In [8]:
 def train_one_epoch(optimizer, scheduler, dataloader):

     model.train()

     train_loss = 0.0

     for batchs in tqdm(dataloader):
         batch = tuple(b.to(device) for b in batchs)

         inputs = {
                 "input_ids": batch[0],
                 "attention_mask": batch[1],
                 "token_type_ids": batch[2],
                 'labels' : batch[3]
             }

         optimizer.zero_grad()

         output = model(**inputs)
         
         loss = output[0]
         
         loss.backward()

         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

         optimizer.step()
         scheduler.step()
        
         train_loss += loss.item()

     avg_train_loss = train_loss / len(dataloader)

     return avg_train_loss

## 2. model evaluation

In [44]:
# 이거로 val_data와 test_data를 평가 가능
def evaluate_one_epoch(dataloader): 
    preds = []
    model.eval()

    eval_loss, eval_accuracy = 0, 0

    for batchs in tqdm(dataloader):
        batch = tuple(b.to(device) for b in batchs)
        inputs = {
                 "input_ids": batch[0],
                 "attention_mask": batch[1],
                 "token_type_ids": batch[2]
             }

        with torch.no_grad():
             output = model(**inputs)
                
        logits = output[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu()
        preds.append(logits)

    return preds

## 3. metric

In [10]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## 4. model

In [72]:
def sentiment_analysis_model(train_data, dev_data, lr=1e-4,epochs = 4, batch_size=32,bert='klue/bert-base', save=True, path='bert_tc'):
    gc.collect()
    torch.cuda.empty_cache()

    model_name = bert.split('/')[-1]

    global tokenizer
    tokenizer = BertTokenizer.from_pretrained(bert)

    global model, device 
    model = BertForSequenceClassification.from_pretrained(bert,num_labels=len(train_data.iloc[:,-1].value_counts()))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    train_dataloader  = preprocess(train_data,batch_size=batch_size, method='train')
    valid_dataloader  = preprocess(dev_data, batch_size=batch_size, method='valid')
    print('')
    print('Data Preprocessing Complete!')


    optimizer = Adam(model.parameters(),
                    lr = lr, # 학습률
                    eps = 1e-8)

    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    print('')
    print('Model Training Start')
    print(f'Epochs : {epochs} / Learning Rate : {str(lr)} / Batch Size : {batch_size}')
    print('')

    loss = []
    acc = []
    for epoch in range(1,epochs+1):
        print(f"epoch = {epoch}")

        train_loss = train_one_epoch(optimizer, scheduler, dataloader=train_dataloader)
        loss.append(train_loss)

        preds = evaluate_one_epoch(dataloader=valid_dataloader)
        preds = (torch.cat(preds, dim=0).argmax(dim=-1)).tolist()
        val_acc = accuracy_score(dev_data['labels'].tolist(),preds)
        acc.append(val_acc)
        
        print(f"Loss = {train_loss:.3f} / Accuracy = {val_acc:.3f}")
        print('')

        if save:
            model.save_pretrained(f'models/{path}')
            tokenizer.save_pretrained(f'models/{path}')
            print('Model Save')

    print('')
    print("Training Complete!")

    return {'loss':loss,'acc':acc}

In [ ]:
# bert = 'klue/bert-base'
# bert = 'beomi/kcbert-base'

score = sentiment_analysis_model(train_data, dev_data,lr=1e-5,batch_size=64 ,epochs = 4, bert = 'klue/bert-base', path='nsmc_test')

# 4. Test

In [66]:
def Sentiment_Analysis(test_data, batch_size=32,bert='nsmc_test'):
    global tokenizer
    tokenizer = BertTokenizer.from_pretrained(f'klue/bert-base')
    test_dataloader = preprocess(test_data, batch_size=batch_size, method='test')

    model = BertForSequenceClassification.from_pretrained(f'models/{bert}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    pred_labels = evaluate_one_epoch(dataloader=test_dataloader)
    pred_labels = (torch.cat(pred_labels, dim=0).argmax(dim=-1)).tolist()

    new_df = test_data[['document_1']].copy()
    new_df['pred_labels'] = pred_labels
    
    return  new_df

In [ ]:
# test_data = 

In [67]:
df = Sentiment_Analysis(dev_data[:10000], batch_size=32,bert='nsmc_test')
df.sample(5)

100%|██████████| 313/313 [01:20<00:00,  3.87it/s]
